# Configuration Validation

> JSON Schema validation helpers for plugin configuration

In [ ]:
#| default_exp utils.validation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Tuple, Optional

try:
    import jsonschema
    HAS_JSONSCHEMA = True
except ImportError:
    HAS_JSONSCHEMA = False

## Validation Functions

These functions provide JSON Schema validation with graceful fallback when `jsonschema` is not available.

In [ ]:
#| export
def validate_config(
    config:Dict[str, Any], # Configuration to validate
    schema:Dict[str, Any] # JSON Schema to validate against
) -> Tuple[bool, Optional[str]]: # (is_valid, error_message)
    """Validate a configuration dictionary against a JSON Schema."""
    # If jsonschema is available, use it for validation
    if HAS_JSONSCHEMA:
        try:
            jsonschema.validate(instance=config, schema=schema)
            return True, None
        except jsonschema.exceptions.ValidationError as e:
            return False, str(e)
        except Exception as e:
            return False, f"Validation error: {str(e)}"
    else:
        # Basic validation without jsonschema
        return _basic_validate(config, schema)

Uses the `jsonschema` library for full validation if available, otherwise falls back to basic validation.

The basic validation (when `jsonschema` is not installed) provides minimal support checking:
- Required fields
- Field types
- Enum values
- Numeric constraints (minimum, maximum)

In [ ]:
#| export
def _basic_validate(
    config:Dict[str, Any], # Configuration to validate
    schema:Dict[str, Any] # JSON Schema to validate against
) -> Tuple[bool, Optional[str]]: # (is_valid, error_message)
    """Basic validation without jsonschema library."""
    try:
        # Check required fields
        required_fields = schema.get("required", [])
        for field in required_fields:
            if field not in config:
                return False, f"Missing required field: {field}"

        # Check field types if properties are defined
        properties = schema.get("properties", {})
        for key, value in config.items():
            if key in properties:
                prop_schema = properties[key]

                # Check enum values
                if "enum" in prop_schema and value not in prop_schema["enum"]:
                    return False, f"Invalid value for {key}: {value}. Must be one of {prop_schema['enum']}"

                # Basic type checking
                expected_type = prop_schema.get("type")
                if expected_type:
                    # Handle nullable types (e.g., ["string", "null"])
                    if isinstance(expected_type, list):
                        if value is None and "null" in expected_type:
                            continue
                        # Get the non-null type
                        types = [t for t in expected_type if t != "null"]
                        if types:
                            expected_type = types[0]

                    type_map = {
                        "string": str,
                        "number": (int, float),
                        "integer": int,
                        "boolean": bool,
                        "array": list,
                        "object": dict
                    }
                    expected_python_type = type_map.get(expected_type)
                    if expected_python_type and not isinstance(value, expected_python_type):
                        return False, f"Invalid type for {key}: expected {expected_type}, got {type(value).__name__}"

                # Check numeric constraints
                if isinstance(value, (int, float)):
                    if "minimum" in prop_schema and value < prop_schema["minimum"]:
                        return False, f"Value for {key} is below minimum: {value} < {prop_schema['minimum']}"
                    if "maximum" in prop_schema and value > prop_schema["maximum"]:
                        return False, f"Value for {key} is above maximum: {value} > {prop_schema['maximum']}"

        return True, None
    except Exception as e:
        return False, f"Validation error: {str(e)}"

In [ ]:
#| export
def extract_defaults(
    schema:Dict[str, Any] # JSON Schema
) -> Dict[str, Any]: # Default values from schema
    """Extract default values from a JSON Schema."""
    defaults = {}

    properties = schema.get("properties", {})
    for key, prop_schema in properties.items():
        if "default" in prop_schema:
            defaults[key] = prop_schema["default"]

    return defaults

### Example: Validating Configuration

In [ ]:
import json

# Define a schema
schema = {
    "type": "object",
    "properties": {
        "model": {
            "type": "string",
            "enum": ["tiny", "base", "small", "medium", "large"],
            "default": "base",
            "description": "Model size to use"
        },
        "temperature": {
            "type": "number",
            "minimum": 0.0,
            "maximum": 1.0,
            "default": 0.0
        },
        "batch_size": {
            "type": "integer",
            "minimum": 1,
            "maximum": 32,
            "default": 8
        }
    },
    "required": ["model"]
}

print("Schema:")
print(json.dumps(schema, indent=2))

Schema:
{
  "type": "object",
  "properties": {
    "model": {
      "type": "string",
      "enum": [
        "tiny",
        "base",
        "small",
        "medium",
        "large"
      ],
      "default": "base",
      "description": "Model size to use"
    },
    "temperature": {
      "type": "number",
      "minimum": 0.0,
      "maximum": 1.0,
      "default": 0.0
    },
    "batch_size": {
      "type": "integer",
      "minimum": 1,
      "maximum": 32,
      "default": 8
    }
  },
  "required": [
    "model"
  ]
}


In [ ]:
# Test valid configurations
valid_configs = [
    {"model": "tiny"},
    {"model": "base", "temperature": 0.5},
    {"model": "large", "batch_size": 16}
]

print("\nValidating valid configurations:")
for config in valid_configs:
    is_valid, error = validate_config(config, schema)
    print(f"Config: {config}")
    print(f"  Valid: {is_valid}")
    if error:
        print(f"  Error: {error}")


Validating valid configurations:
Config: {'model': 'tiny'}
  Valid: True
Config: {'model': 'base', 'temperature': 0.5}
  Valid: True
Config: {'model': 'large', 'batch_size': 16}
  Valid: True


In [ ]:
# Test invalid configurations
invalid_configs = [
    ({"temperature": 0.5}, "Missing required 'model' field"),
    ({"model": "invalid"}, "Invalid enum value"),
    ({"model": "base", "temperature": 1.5}, "Temperature exceeds maximum"),
    ({"model": "base", "batch_size": 100}, "Batch size exceeds maximum")
]

print("\nValidating invalid configurations:")
for config, description in invalid_configs:
    is_valid, error = validate_config(config, schema)
    print(f"\n{description}:")
    print(f"  Config: {config}")
    print(f"  Valid: {is_valid}")
    if error:
        print(f"  Error: {error[:100]}...")  # Truncate long errors


Validating invalid configurations:

Missing required 'model' field:
  Config: {'temperature': 0.5}
  Valid: False
  Error: 'model' is a required property

Failed validating 'required' in schema:
    {'type': 'object',
     ...

Invalid enum value:
  Config: {'model': 'invalid'}
  Valid: False
  Error: 'invalid' is not one of ['tiny', 'base', 'small', 'medium', 'large']

Failed validating 'enum' in sc...

Temperature exceeds maximum:
  Config: {'model': 'base', 'temperature': 1.5}
  Valid: False
  Error: 1.5 is greater than the maximum of 1.0

Failed validating 'maximum' in schema['properties']['tempera...

Batch size exceeds maximum:
  Config: {'model': 'base', 'batch_size': 100}
  Valid: False
  Error: 100 is greater than the maximum of 32

Failed validating 'maximum' in schema['properties']['batch_si...


In [ ]:
# Test extracting defaults
defaults = extract_defaults(schema)
print("\nDefault values extracted from schema:")
print(json.dumps(defaults, indent=2))


Default values extracted from schema:
{
  "model": "base",
  "temperature": 0.0,
  "batch_size": 8
}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()